#### Импорт необходимых библиотек

In [4]:
import numpy as np
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import speech_recognition as sr
import soundfile as sf
import os

# 1. Функция для загрузки шумовых образцов
def load_noise_samples(noise_dir):
    """Загрузить и объединить шумовые образцы в один аудиофайл"""
    noise_files = [os.path.join(noise_dir, f) for f in os.listdir(noise_dir) if f.endswith('.wav')]
    noises = []
    for noise_file in noise_files:
        noise, sr = librosa.load(noise_file, sr=16000)
        noises.append(noise)
    combined_noise = np.concatenate(noises)
    return combined_noise, sr

# 2. Функция для удаления шума
def reduce_noise(audio_file, noise_dir, output_file, noise_reduction_factor=1):
    """
    Удалить шум из голосового сообщения.
    """
    # Загрузить аудиофайл с голосом
    voice, sr = librosa.load(audio_file, sr=16000)

    # Загрузить шумы
    noise, sr_noise = load_noise_samples(noise_dir)

    if sr != sr_noise:
        raise ValueError("Частоты дискретизации аудиофайла и шума не совпадают")

    # Убедиться, что длина шума соответствует длине аудио (или короче)
    noise = noise[:len(voice)]

    # Преобразование аудио и шума в STFT (Short-Time Fourier Transform)
    voice_stft = librosa.stft(voice)
    noise_stft = librosa.stft(noise)

    # Усреднить шумы по времени
    noise_mean = np.mean(np.abs(noise_stft), axis=1)

    # Вычесть шум из голоса
    voice_clean_stft = voice_stft - noise_reduction_factor * noise_mean[:, np.newaxis]

    # Преобразовать обратно в аудиосигнал
    voice_clean = librosa.istft(voice_clean_stft)

    # Сохранить очищенный аудиофайл
    sf.write(output_file, voice_clean, sr)

    print(f"Шум успешно удален и файл сохранен в {output_file}")
    return output_file

# 3. Функция для преобразования речи в текст
def speech_to_text(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data, language='ru-RU')
            return text
        except sr.UnknownValueError:
            return "Не удалось распознать речь"
        except sr.RequestError as e:
            return f"Ошибка сервиса: {e}"

# 4. Функция для создания модели классификации текста
def build_text_classifier(num_classes):
    model = keras.Sequential([
        layers.Embedding(input_dim=1000, output_dim=64),
        layers.GlobalAveragePooling1D(),
        layers.Dense(16, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Предположим, num_classes категорий
    ])

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

# 5. Основная функция обработки аудиофайла
def process_audio_file(audio_path, noise_dir, categories):
    # Удаление шума
    cleaned_audio = reduce_noise(audio_path, noise_dir, 'cleaned_voice_message.wav')

    # Преобразование речи в текст
    text = speech_to_text(cleaned_audio)
    print("Распознанный текст:", text)

    # Преобразование текста в массив для классификации
    text_input = np.array([text])  # Пример преобразования текста в массив

    # Создание и обучение модели (или загрузка предварительно обученной модели)
    model = build_text_classifier(num_classes=len(categories))

    # Здесь должен быть код для загрузки и подготовки данных для обучения
    # model.fit(training_data, training_labels, epochs=10)  # Пример обучения

    # Предсказание категории
    prediction = model.predict(text_input)
    predicted_category = categories[np.argmax(prediction)]
    print("Предсказанная категория:", predicted_category)

    return model  # Вернуть модель для сохранения

# 6. Сохранение модели в формате TFLite
def save_model_to_tflite(model, filename):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open(filename, 'wb') as f:
        f.write(tflite_model)

# Использование
audio_file_path = "path/to/your/audio/file.wav"
noise_directory = "path/to/your/noise/files"
categories = [
    "отказ",
"отмена",
"подтверждение",
"начать осаживание",
"осадить на (количество) вагон",
"продолжаем осаживание",
"зарядка тормозной магистрали",
"вышел из межвагонного пространства",
"продолжаем роспуск",
"растянуть автосцепки",
"протянуть на (количество) вагон",
"отцепка",
"назад на башмак",
"захожу в межвагонное пространство",
"остановка",
"вперед на башмак",
"сжать автосцепки",
"назад с башмака",
"тише",
"вперед с башмака",
"прекратить зарядку тормозной магистрали",
"тормозить",
"отпустить",
]


# Обработка аудиофайла и получение модели
model = process_audio_file(audio_file_path, noise_directory, categories)

# Сохранение модели в TFLite формат
save_model_to_tflite(model, 'text_classifier_model.tflite')
